In [1]:
import aocd
from math import sqrt

In [2]:
def parse2(chunk):
    warehouse = dict()
    for real, line in enumerate(chunk.splitlines()):
        for imag, char in enumerate(line):
            position = complex(real, 2*imag)
            match char:
                case '#':
                    warehouse[position+0j] = '#'
                    warehouse[position+1j] = '#'
                case 'O':
                    warehouse[position+0j] = '['
                    warehouse[position+1j] = ']'
                case '.':
                    warehouse[position+0j] = '.'
                    warehouse[position+1j] = '.'
                case '@':
                    warehouse[position+0j] = '@'
                    warehouse[position+1j] = '.'
                    robot = position
    return warehouse, robot

In [3]:
data = aocd.get_data()
chunk1, chunk2 = data.split('\n\n')
warehouse, robot = parse2(chunk1)
directions = chunk2.replace('\n', '')

In [4]:
north = lambda x: x-1
south = lambda x: x+1
east  = lambda x: x+1j
west  = lambda x: x-1j

def can_move(position, direction):
    next_pos = {'^': north, 'v': south}[direction](position)
    match warehouse[next_pos]:
        case '#':
            return False
        case '[':
            return can_move(next_pos, direction) and can_move(next_pos+1j, direction)
        case ']':
            return can_move(next_pos, direction) and can_move(next_pos-1j, direction)
        case '.':
            return True

def move(position, direction):
    sprite = warehouse[position]
    next_pos = {'^': north, 'v': south, '<': west, '>': east}[direction](position)
    # print('moving', sprite, 'to', next_pos, warehouse[next_pos])
    match warehouse[next_pos]:
        case '.':
            warehouse[next_pos] = sprite
            warehouse[position] = '.'
            return next_pos
        case '#':
            return position
        case box if (box == 'O') or (box in '[]' and direction in '<>'):
            if move(next_pos, direction) != next_pos:
                return move(position, direction)
            else:
                return position
        case box:
            next_pos2 = next_pos+1j if box == '[' else next_pos-1j
            if can_move(next_pos, direction) and can_move(next_pos2, direction):
                move(next_pos, direction)
                move(next_pos2, direction)
                return move(position, direction)
            else:
                return position

def gps(position):
    return int(100*position.real + position.imag) if warehouse[position] == '[' else 0

def draw():
    dim = int(sqrt(len(warehouse) // 2))
    for real in range(dim):
        for imag in range(2*dim):
            print(warehouse[complex(real, imag)], end='')
        print()

In [5]:
for direction in directions:
    robot = move(robot, direction)

sum(gps(position) for position in warehouse)

1376686